# Agent Based Model: Main Notebook

In [206]:
import numpy.random as rnd
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
import random

#### Workflow Notes 
* Draw Students with values within the four dimensions 
* Function that creates a study group (draw 4 students) -> Measure homogeniety 
* Task 


## Creating Study Groups

#### Practical Functions

In [170]:
random.seed(1)
def n_sampler(size):
    '''
    sample a number of random numbers from the beta distribution
    '''
    max_vals = []
    min_vals = []

    beta_dist = rnd.beta(2.5, 2.5, size)

    for i in range(size):
        if beta_dist[i] >= 0.5:
            max_vals.append(beta_dist[i])
        else:
            min_vals.append(beta_dist[i])
    
    return max_vals, min_vals

max_vals, min_vals = n_sampler(1000)

In [171]:
random.seed(1)
def data_collect(studygroup, student_list):
    '''
    function to collect data from the simulation
    '''
    ID_list = []
    extraversion_list = []
    sensing_list = []
    thinking_list = []
    judging_list = []
    academic_list = [] 
    social_list = []
    
    for student in studygroup:
        ID_list.append(student.ID)
        extraversion_list.append(student.ExScore)
        sensing_list.append(student.SeScore) 
        thinking_list.append(student.ThScore)
        judging_list.append(student.JuScore)
        academic_list.append(student.Academic_skill)
        social_list.append(student.Social_skill)

    data = pd.DataFrame({'ID': ID_list, 
                        'type': student_list, 
                        'E/I': extraversion_list, 
                        'S/N': sensing_list,
                        'T/F': thinking_list,
                        'J/P': judging_list, 
                        'Academic': academic_list,
                        'Social': social_list})
    
    return data
    

#### Create Study Group

In [172]:
## Defining the Students ##
class Student():
    def __init__(self, ID, Ex, Se, Th, Ju):
        self.ID  = ID
        self.Ex = Ex #Extraversion vs Introversion dimension
        self.Se = Se #Sensing vs Intuition dimension
        self.Th = Th #Thinking vs Feeling dimension
        self.Ju = Ju #Judging vs Perceiving dimension

        self.ExScore = 0
        self.SeScore = 0
        self.ThScore = 0
        self.JuScore = 0
        
        self.Scores = []

        self.Academic_Skill = 0
        self.Social_Skill = 0

        self.Own_solution = []


def personality(student):
    # Extraversion vs. Introversion
    if student.Ex == "E":
        student.ExScore = max_vals[0]
        del max_vals[0]
    else:
        student.ExScore = min_vals[0]
        del min_vals[0]
    
    # Sensing vs. Intuition
    if student.Se == "S":
        student.SeScore = max_vals[0]
        del max_vals[0]
    else:
        student.SeScore = min_vals[0]
        del min_vals[0]
    
    # Thinking vs. Feeling
    if student.Th == "T":
        student.ThScore = max_vals[0]
        del max_vals[0]
    else:
        student.ThScore = min_vals[0]
        del min_vals[0]

    # Judging vs. Perceiving
    if student.Ju == "J":
        student.JuScore = max_vals[0]
        del max_vals[0]
    else:
        student.JuScore = min_vals[0]
        del min_vals[0]
    
    student.Scores = [student.ExScore, student.SeScore, student.ThScore, student.JuScore]

def skills(student):
    student.Academic_skill = (1-student.ExScore)*0.15 + (1-student.SeScore)*0.35 + student.ThScore*0.20 + (1-student.JuScore)*0.30
    student.Social_skill = student.ExScore*0.50 + (1-student.SeScore)*0.10 + student.ThScore*0.30 + (1-student.JuScore)*0.10

In [176]:
def StudyGroup(student_list):
    '''
    Create a study group of students
    '''
    studygroup = []

    for i in range(len(student_list)):
        student = Student(i+1, student_list[i][0], student_list[i][1], student_list[i][2], student_list[i][3])
        personality(student)
        skills(student)
        studygroup.append(student)
    
    return data_collect(studygroup, student_list), studygroup

In [177]:
random.seed(1)
student_list_test = ["ESTJ", "ISTJ", "ENTP", "INFJ"]

studygroup1 = StudyGroup(student_list_test)


print(studygroup1[0])

   ID  type       E/I       S/N       T/F       J/P  Academic    Social
0   1  ESTJ  0.751757  0.680962  0.647226  0.551739  0.412823  0.646777
1   2  ISTJ  0.431339  0.653384  0.604921  0.781501  0.393149  0.453657
2   3  ENTP  0.718741  0.459212  0.788625  0.353230  0.583221  0.714714
3   4  INFJ  0.405175  0.309533  0.397507  0.521759  0.553861  0.438710


In [ ]:
student_list_test_2 = ["ESTJ", "ISTJ", "ENTP", "INFJ"]
studygroup2 = StudyGroup(student_list_test_2)


   ID  type       E/I       S/N       T/F       J/P  Academic    Social
0   1  ESTJ  0.748458  0.557823  0.608510  0.583156  0.439248  0.642684
1   2  ISTJ  0.077268  0.637742  0.809514  0.858011  0.469700  0.331913
2   3  ENTP  0.514659  0.247829  0.602677  0.335023  0.656089  0.579848
3   4  INFJ  0.271669  0.208121  0.329171  0.691234  0.544871  0.344650


In [180]:
print(studygroup2[0])

   ID  type       E/I       S/N       T/F       J/P  Academic    Social
0   1  ESTJ  0.748458  0.557823  0.608510  0.583156  0.439248  0.642684
1   2  ISTJ  0.077268  0.637742  0.809514  0.858011  0.469700  0.331913
2   3  ENTP  0.514659  0.247829  0.602677  0.335023  0.656089  0.579848
3   4  INFJ  0.271669  0.208121  0.329171  0.691234  0.544871  0.344650


## Creating the ABM

### Generating True Solution

In [181]:
def true_solution_generator(n_elements, range_elements):
    '''
    create a list of random numbers that will serve as the true solution that the agents need to find 
    '''
    true_solution = []
    for i in range(n_elements):
        true_solution.append(random.randint(range_elements[0], range_elements[1]))

    return true_solution, range_elements

In [182]:
true_solution = true_solution_generator(8, [1,9])
print(true_solution[0])

[3, 2, 5, 2, 8, 8, 8, 7]


### Generating Individual Solutions

In [237]:
def individual_solutions_generator(studygroup, true_solution, range_elements):
    '''
    function to calculate the individual solutions of the agents given a study group dataframe and a true solution
    '''
    
    for student in studygroup[1]: # loop for each individual
        Own_solution_lst = []
        
        for i in range(len(true_solution)): # loop for each part-exercise
            coin_toss = np.random.binomial(1, student.Academic_skill, 1)[0] # biased-coin flip
            if coin_toss == 1:
                Own_solution_lst.append(true_solution[i])
            else:
                Own_solution_lst.append(random.randint(range_elements[0], range_elements[1]))
        
        student.Own_solution = Own_solution_lst

In [238]:
individual_solutions_generator(studygroup1, true_solution[0], true_solution[1])

In [240]:
[student.Own_solution for student in studygroup1[1]]

[[4, 2, 5, 1, 8, 8, 2, 7],
 [3, 3, 5, 3, 8, 8, 9, 7],
 [4, 2, 5, 5, 6, 9, 5, 7],
 [3, 6, 6, 6, 8, 8, 2, 5]]

## Collaborative Problem Solving

In [ ]:
max_ticks = 1000 #turns in the game


##### Workflow notes
1. Who presents their solution e.g. agent A presents their solution to agent B, C, D -> THE PROPOSED SOLUTION
    -> Based on Extraversion score (Highest extraversion score is the most likely to present their solution)
2. According to an Agreeableness score of the other agents (and maybe a Trustworthiness score of agent proposing), agents will update their solution
3. The solutions of the agents will be checked, if all they agree, this is their final solution. If not, the process will be repeated for a max of X ticks. If the groups do not converge, an accuracy score will still be calculated. 